In [24]:
import pandas as pd


# Display or use the filtered DataFrame as needed
input_file_path = 'final_output/final_raw_dataset.xlsx'
ndf = pd.read_excel(input_file_path)
# coding all brandname with data less than 50 to Other or REMOVING ALL OTHER BRANDS

def checkValueCount(dataset,Column):
    valueCount=dataset.value_counts(Column)
    return valueCount
brandFinal=checkValueCount(ndf,'BRANDNAME')
# print(brandFinal);
brands_to_keep = brandFinal[brandFinal > 50].index.tolist()
#THIS CODE HELPS TO CODE BRAND WITH VALUE LESS THAN 50 TO OTHER BRAND
ndf['BRANDNAME'] = ndf['BRANDNAME'].apply(lambda x: x if x in brands_to_keep else 'Other Brands')

#//THIS CODE REMOVES ALL DATA WITH BRAND COUNT  LESS THAN 50
# dfNew = ndf[ndf['BRANDNAME'].isin(brands_to_keep)]


# ndf.info();


In [ ]:
valueCount=ndf.value_counts('BODY')
print(valueCount)
ndf['BODY'] = ndf['BODY'].replace({r'.*SUV.*': 'SUV', r'.*Hatchback.*': 'Hatchback'}, regex=True, inplace=False)
#Removing  row of data with count of body type  less than 50
# ndf = ndf[nd?f['BODY'].map(ndf['BODY'].value_counts()) >= 50]

In [ ]:
# coding all Color with data less than 100 to Other and leaving rest as it is to check impact of color

valueCount=ndf.value_counts('COLOR')
colorsToKeep = valueCount[valueCount > 50].index.tolist()
ndf['COLOR'] = ndf['COLOR'].replace('Other', 'Other Colors')
ndf['COLOR'] = ndf['COLOR'].apply(lambda x: x if x in colorsToKeep else 'Other Colors')

print(ndf.value_counts('COLOR'))




In [ ]:

#Creating dummy variables for brand , color and body type,
# Also creating dummy variables for binary data like Fuel, Drive, Gearbox
import os;

normalDf=ndf.copy()
from sklearn.preprocessing import LabelEncoder
output_dir='final_output'
output_file_path = os.path.join(output_dir, 'full_dummy_data.csv')

label_encoder = LabelEncoder()
normalDf['FUEL'] = label_encoder.fit_transform(normalDf['FUEL'])
normalDf['DRIVE'] = label_encoder.fit_transform(normalDf['DRIVE'])
normalDf['GEARBOX'] = label_encoder.fit_transform(normalDf['GEARBOX'])
normalDf=normalDf.join(pd.get_dummies(normalDf.BRANDNAME)).drop(['BRANDNAME'], axis=1);
normalDf=normalDf.join(pd.get_dummies(normalDf.COLOR)).drop(['COLOR'], axis=1)  ;
normalDf=normalDf.join(pd.get_dummies(normalDf.BODY)).drop(['BODY'], axis=1);
normalDf=normalDf.join(pd.get_dummies(normalDf.ENGINERANGE)).drop(['ENGINERANGE'], axis=1);
normalDf = normalDf.astype(int)

normalDf.to_csv(output_file_path, index=False)

normalDf.head()

In [42]:
#Running OLS summary for Normal Regression
# Import necessary libraries
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.diagnostic import het_breuschpagan

# Set display options for pandas
pd.set_option('display.max_rows', None)

# Copy the original DataFrame
data = normalDf.copy()

# Define dependent variable
dependent_variable = 'PRICE'

# Select all columns except the dependent variable for X
X = data.drop(columns=[dependent_variable])

# Display the first few rows of the DataFrame

# Define the dependent variable
y = data[dependent_variable]

# Print summary for normal regression
print('***********************Normal Regression Summary*************************')

# Add a constant term for the intercept
X_normal = sm.add_constant(X)

# Fit the OLS model
model_normal = sm.OLS(y, X_normal).fit()

# Print the summary of the model
print(model_normal.summary())

# Get residuals
residuals = model_normal.resid

# Optional: Breusch-Pagan test for heteroskedasticity
bp_test = het_breuschpagan(residuals, X_normal)
bp_results = pd.DataFrame(bp_test, index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']).T
print('***********************Breusch-Pagan Test Results*************************')
print(bp_results)

***********************Normal Regression Summary*************************
                            OLS Regression Results                            
Dep. Variable:                  PRICE   R-squared:                       0.486
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     100.6
Date:                Mon, 30 Sep 2024   Prob (F-statistic):               0.00
Time:                        16:21:28   Log-Likelihood:                -59361.
No. Observations:                3766   AIC:                         1.188e+05
Df Residuals:                    3730   BIC:                         1.190e+05
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------

In [40]:
#Running OLS summary for Semi log
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.diagnostic import het_breuschpagan

pd.set_option('display.max_rows', None)

data=normalDf.copy()
dependent_variable = 'PRICE'
# columns_to_use = ['MILEAGE', 'FUEL','DRIVE','GEARBOX', 'AGE','Ford','Hyundai','Kia','Mahindra','Maruti Suzuki','Nissan','Tata','Toyota','Volkswagen','Black','Gray','Blue','Brown','Maroon','Red','Silver','Other Colors','Hatchback','Pickup','SUV','Sedan','1001-1500(cc)','1501-2000(cc)','2001-2500(cc)','2501-2900(cc)','2901-above(cc)']

data.head()
y = data[dependent_variable]

# Select all columns except the dependent variable for X
X = data.drop(columns=[dependent_variable])



print('***********************Semi Log Summary*************************')
y_semi_log = np.log(y)
X_semi_log=X.copy()
X_semi_log = sm.add_constant(X_semi_log)  # Add a constant term for the intercept

model_semi_log = sm.OLS(y_semi_log, X_semi_log).fit()
print(model_semi_log.summary())
# print(model_semi_log.bse)
residuals = model_semi_log.resid

***********************Semi Log Summary*************************
                            OLS Regression Results                            
Dep. Variable:                  PRICE   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     252.0
Date:                Mon, 30 Sep 2024   Prob (F-statistic):               0.00
Time:                        16:21:04   Log-Likelihood:                -1616.3
No. Observations:                3766   AIC:                             3305.
Df Residuals:                    3730   BIC:                             3529.
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

In [43]:
print('Box COX Implementation')
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from statsmodels.stats.diagnostic import het_breuschpagan
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment

# Assuming 'data' is your DataFrame
data = normalDf.copy()

# Define the dependent and independent variables
dependent_variable = 'PRICE'

# Apply Box-Cox transformation to the dependent variable
y_transformed, lambda_value = stats.boxcox(data[dependent_variable])

print('*****lambda'+str(lambda_value)+'*****')
# Update the dependent variable in the DataFrame with the transformed values
data[dependent_variable] = y_transformed

# Separate the dependent and independent variables
y = data[dependent_variable]
X = data.drop(columns=[dependent_variable])


# Add a constant term for the intercept
X_linear = sm.add_constant(X)

# Fit the linear regression model with the transformed dependent variable
print('***********************Box-Cox Transformed Summary*************************')
model_boxcox = sm.OLS(y, X_linear).fit()
print(model_boxcox.summary())



print('*************Export to excel the summary***************')

excel_file = 'linear_regression_summary.xlsx'
writer = pd.ExcelWriter(excel_file, engine='openpyxl')
summary = model_boxcox.summary2().tables
# Initialize a workbook and grab the active worksheet
workbook = Workbook()
worksheet = workbook.active
worksheet.title = 'Summary'

# Define a starting row
start_row = 1

# Function to write DataFrame to Excel starting from a specific row
def write_table_to_excel(table, start_row):
    df = pd.DataFrame(table)
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=False), start=start_row):
        for c_idx, value in enumerate(row, start=1):
            worksheet.cell(row=r_idx, column=c_idx, value=value)
            worksheet.cell(row=r_idx, column=c_idx).alignment = Alignment(horizontal='center', vertical='center')

# Write each table in the summary to Excel
for table in summary:
    write_table_to_excel(table, start_row)
    start_row += len(table) + 2  # Add some space between tables

# Save the Excel file
workbook.save('linear_regression_summary.xlsx')
print('*************End of Export to excel the summary***************')

predicted_boxcox = model_boxcox.predict(X_linear)

# Step 2: Transform predicted values back to original scale
predicted_original_scale = (np.power((predicted_boxcox * lambda_value) + 1, 1 / lambda_value))

# Step 3: Calculate residuals in original scale
residuals = data[dependent_variable] - predicted_original_scale

lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(residuals, X_linear)

print("Breusch-Pagan test: Box Cox Transformation ")
print("LM Statistic:", lm)
print("LM Test p-value:", lm_p_value)
print("F-Statistic:", fvalue)
print("F-Test p-value:", f_p_value)

# Step 4: Calculate RMSE
rmse_boxcox = np.sqrt(np.mean(residuals**2))

print("RMSE for Box-Cox transformed model:", rmse_boxcox)

Box COX Implementation
*****lambda-0.09271048522907309*****
***********************Box-Cox Transformed Summary*************************
                            OLS Regression Results                            
Dep. Variable:                  PRICE   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     255.0
Date:                Mon, 30 Sep 2024   Prob (F-statistic):               0.00
Time:                        16:21:57   Log-Likelihood:                 3458.2
No. Observations:                3766   AIC:                            -6844.
Df Residuals:                    3730   BIC:                            -6620.
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025    

https://www.youtube.com/watch?v=Wqmtf9SA_kk&ab_channel=NeuralNine
https://www.youtube.com/watch?v=xu0prYu5e-c&ab_channel=AliNasserEddine